# 1. Exploring Data Sources and Use Cases

## Initial Data Exploration

### The use-case to implement is collected from https://www.kaggle.com/datasets - flower-recognition dataset

In [2]:
# The code was removed by Watson Studio for sharing.

# Extract Transform Load (ETL)

In [3]:
from io import BytesIO
import os
import zipfile

zip_ref = zipfile.ZipFile(BytesIO(streaming_body_2.read()), 'r')

zip_ref .extractall("../input/flower5/")
#zip_ref.close()

In [4]:
data = "../input/flower5/flowers"

# List out the directories inside the main input folder

folders = os.listdir(data)

print(folders)

['sunflower', 'dandelion', 'rose', 'daisy', 'tulip']


In [5]:
!conda install -c conda-forge scikit-image

Solving environment: done

## Package Plan ##

  environment location: /home/spark/shared/conda/envs/python3.6

  added / updated specs:
    - scikit-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    cloudpickle-1.2.2          |             py_1          23 KB  conda-forge
    cycler-0.10.0              |             py_2           9 KB  conda-forge
    cytoolz-0.10.1             |   py36h516909a_0         431 KB  conda-forge
    dask-core-2.9.0            |             py_0         581 KB  conda-forge
    dbus-1.13.6                |       he372182_0         602 KB  conda-forge
    decorator-4.4.1            |             py_0          11 KB  conda-forge
    expat-2.2.5                |    he1b5a44_1004         191 KB  conda-fo

# Feature Creation

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
from skimage.io import imread
from skimage import transform as tf
import pandas
import os
import random
image_names1 = []
train_labels1 = []
train_images1 = []

size = 28,28

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names1.append(os.path.join(data,folder,file))
            train_labels1.append(folder)
            img = imread(os.path.join(data,folder,file))
            im = tf.resize(img,size)
            train_images1.append(im)
        else:
            continue

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(["tulip", "dandelion", "sunflower", "daisy", "rose"])


LabelEncoder()

In [10]:
train = np.array(train_images1)



train = train.astype('float32') / 255.0


labels =  le.transform(train_labels1)


# Convert the  list to numpy array type

train_image = np.array(train)
labels_image = np.array(labels)


In [42]:

from sklearn.model_selection import train_test_split

train_images, test_images,train_labels, test_labels = train_test_split(train_image,labels_image, test_size=0.2,shuffle=True)


array([1, 4, 3, 4, 1, 0, 2, 0, 4])

In [12]:
!pip install tensorflow==1.14.0

    100% |################################| 109.2MB 150kB/s eta 0:00:01K    34% |###########                     | 38.2MB 69.1MB/s eta 0:00:02[K    45% |##############                  | 49.3MB 36.0MB/s eta 0:00:02    54% |#################               | 60.0MB 10.4MB/s eta 0:00:05    57% |##################              | 62.3MB 64.2MB/s eta 0:00:01
    100% |################################| 112kB 2.8MB/s eta 0:00:01
    100% |################################| 51kB 1.9MB/s eta 0:00:01
    100% |################################| 3.2MB 2.2MB/s eta 0:00:01
    100% |################################| 491kB 2.9MB/s eta 0:00:01
    100% |################################| 2.4MB 1.8MB/s eta 0:00:01
    100% |################################| 51kB 1.8MB/s eta 0:00:01
    100% |################################| 1.3MB 2.6MB/s eta 0:00:01
    100% |################################| 61kB 1.8MB/s eta 0:00:01
    100% |################################| 20.0MB 684kB/s eta 0:00:01
    100% |#######

In [13]:

import tensorflow as tf


In [14]:
test_images[0].shape

(28, 28, 3)

# Model Definition and Training

In [83]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.nn import relu
from tensorflow.nn import softmax
from tensorflow.nn import sigmoid

    
model = Sequential([
    Flatten(input_shape=(28, 28,3)),
    Dense(512, activation=relu),
    Dense(5, activation=softmax)
])

model.compile(optimizer='adam', 
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=50, verbose=0)

model.evaluate(test_images, test_labels)

865/865 [==============================] - 0s 63us/sample - loss: 1.1794 - acc: 0.4925


[1.1794200982661605, 0.49248555]

# Model Evaluation, Tuning, Deployment and Documentation

In [60]:
!wget https://images.all-free-download.com/images/graphiclarge/sunflower_05_hd_picture_167005.jpg

--2019-12-16 20:46:19--  https://images.all-free-download.com/images/graphiclarge/sunflower_05_hd_picture_167005.jpg
Resolving images.all-free-download.com (images.all-free-download.com)... 207.182.153.238
Connecting to images.all-free-download.com (images.all-free-download.com)|207.182.153.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37349 (36K) [image/jpeg]
Saving to: 'sunflower_05_hd_picture_167005.jpg.2'

100%[======================================>] 37,349      --.-K/s   in 0.03s   

2019-12-16 20:46:20 (1.09 MB/s) - 'sunflower_05_hd_picture_167005.jpg.2' saved [37349/37349]



In [61]:
!mv sunflower_05_hd_picture_167005.jpg sunflower.jpg

In [62]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


img_path = 'sunflower.jpg'
img = image.load_img(img_path, target_size=(28, 28))
x = image.img_to_array(img)
x.shape
#img = x.astype('float32') / 255.0
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [84]:
preds = model.predict(x)

In [85]:
preds


array([[0., 1., 0., 0., 0.]], dtype=float32)

In [99]:
position = np.argmax(preds)

In [102]:
print('Predicted:', le.inverse_transform([position]))

Predicted: ['dandelion']


In [118]:
!wget https://upload.wikimedia.org/wikipedia/commons/5/51/Small_Red_Rose.JPG

--2019-12-16 22:40:48--  https://upload.wikimedia.org/wikipedia/commons/5/51/Small_Red_Rose.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1380091 (1.3M) [image/jpeg]
Saving to: 'Small_Red_Rose.JPG'

100%[======================================>] 1,380,091   4.73MB/s   in 0.3s   

2019-12-16 22:40:48 (4.73 MB/s) - 'Small_Red_Rose.JPG' saved [1380091/1380091]



In [123]:
!mv Small_Red_Rose.JPG rose.jpg

In [124]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


img_path = 'rose.jpg'
img = image.load_img(img_path, target_size=(28, 28))
x = image.img_to_array(img)
x.shape
#img = x.astype('float32') / 255.0
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
position = np.argmax(preds)
print('Predicted:', le.inverse_transform([position]))

Predicted: ['rose']


In [128]:
!wget  https://images.all-free-download.com/images/graphiclarge/dandelion_204752.jpg

--2019-12-16 22:44:07--  https://images.all-free-download.com/images/graphiclarge/dandelion_204752.jpg
Resolving images.all-free-download.com (images.all-free-download.com)... 207.182.153.238
Connecting to images.all-free-download.com (images.all-free-download.com)|207.182.153.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61572 (60K) [image/jpeg]
Saving to: 'dandelion_204752.jpg'

100%[======================================>] 61,572      --.-K/s   in 0.07s   

2019-12-16 22:44:08 (857 KB/s) - 'dandelion_204752.jpg' saved [61572/61572]



In [129]:
!mv dandelion_204752.jpg dandelion.jpg

In [130]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


img_path = 'dandelion.jpg'
img = image.load_img(img_path, target_size=(28, 28))
x = image.img_to_array(img)
x.shape
#img = x.astype('float32') / 255.0
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
position = np.argmax(preds)
print('Predicted:', le.inverse_transform([position]))

Predicted: ['dandelion']


In [140]:
!wget https://images.all-free-download.com/images/graphiclarge/the_fragant_daisy_514090.jpg

--2019-12-16 22:48:41--  https://images.all-free-download.com/images/graphiclarge/the_fragant_daisy_514090.jpg
Resolving images.all-free-download.com (images.all-free-download.com)... 207.182.153.238
Connecting to images.all-free-download.com (images.all-free-download.com)|207.182.153.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75460 (74K) [image/jpeg]
Saving to: 'the_fragant_daisy_514090.jpg'

100%[======================================>] 75,460      --.-K/s   in 0.1s    

2019-12-16 22:48:41 (653 KB/s) - 'the_fragant_daisy_514090.jpg' saved [75460/75460]



In [141]:
!mv the_fragant_daisy_514090.jpg daisy.jpg

In [142]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


img_path = 'daisy.jpg'
img = image.load_img(img_path, target_size=(28, 28))
x = image.img_to_array(img)
x.shape
#img = x.astype('float32') / 255.0
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
position = np.argmax(preds)
print('Predicted:', le.inverse_transform([position]))

Predicted: ['dandelion']
